In [14]:
gap_score = -10
aa_pairs = dict()
f = open('BLOSUM62.txt','r')
for line in f:
    if line.startswith('#'):
        continue
    elif line.startswith(' '):
        headers = line.strip().split()
    else:
        tmp_list = line.strip().split()
        aa_pairs[tmp_list[0]] = dict()
        for i in range(1,len(tmp_list)):
            aa_pairs[tmp_list[0]][headers[i-1]] = int(tmp_list[i])
f.close()

In [2]:
seq_list = dict()
f_fa = open('insulin_pep.uniprot.fa','r')
for line in f_fa:
    if line.startswith('>'):
        ## >sp|P01308|INS_HUMAN Insulin OS=Homo sapiens GN=INS PE=1 SV=1
        ## --> 'HUMAN'
        seq_h = line.strip().split()[0].split('_')[-1]
        seq_list[seq_h] = ''
    else:
        seq_list[seq_h] += line.strip()
f_fa.close()

In [15]:
import numpy as np

def global_alignment(tmp_matrix, tmp_pos1, tmp_pos2, tmp_chr1, tmp_chr2):    
    if tmp_pos1 == 0 and tmp_pos2 == 0:
        return 's',0
    elif tmp_pos2 == 0:
        return 'v',tmp_matrix[tmp_pos1-1][0] + gap_score
    elif tmp_pos1 == 0:
        return 'h',tmp_matrix[0][tmp_pos2-1] + gap_score
    else:
        d_score = tmp_matrix[tmp_pos1-1][tmp_pos2-1]
        d_score += aa_pairs[tmp_chr1][tmp_chr2]
        v_score = tmp_matrix[tmp_pos1-1][tmp_pos2] + gap_score
        h_score = tmp_matrix[tmp_pos1][tmp_pos2-1] + gap_score
        
        max_score = max([d_score,v_score,h_score])
        tmp_dir_list = []
        if d_score == max_score:
            tmp_dir_list.append('d')
        if v_score == max_score:
            tmp_dir_list.append('v')
        if h_score == max_score:
            tmp_dir_list.append('h')
            
        return ''.join(tmp_dir_list), max_score

def local_alignment(tmp_matrix, tmp_pos1, tmp_pos2, tmp_chr1, tmp_chr2):    
    if tmp_pos1 == 0 or tmp_pos2 == 0:
        return 's',0
    else:
        d_score = tmp_matrix[tmp_pos1-1][tmp_pos2-1]
        d_score += aa_pairs[tmp_chr1][tmp_chr2]
        v_score = tmp_matrix[tmp_pos1-1][tmp_pos2] + gap_score
        h_score = tmp_matrix[tmp_pos1][tmp_pos2-1] + gap_score
        
        max_score = max([d_score,v_score,h_score,0])
        tmp_dir_list = []
        if d_score == max_score:
            tmp_dir_list.append('d')
        if v_score == max_score:
            tmp_dir_list.append('v')
        if h_score == max_score:
            tmp_dir_list.append('h')
        if len(tmp_dir_list) == 0:
            tmp_dir_list.append('s')
            
        return ''.join(tmp_dir_list), max_score
    
def align_pairwise(tmp_seq1, tmp_seq2):
    len_seq1 = len(tmp_seq1)
    len_seq2 = len(tmp_seq2)
    
    dir_list = dict()
    max_score = -100
    (max_pos1, max_pos2) = (0,0)
    score_matrix = np.zeros( (len_seq1+1, len_seq2+1) )
    for pos1 in range(len_seq1+1):
        chr1 = tmp_seq1[pos1-1]
        for pos2 in range(len_seq2+1):
            chr2 = tmp_seq2[pos2-1]
            ## for global alignment
            #tmp_dir, tmp_score = global_alignment(score_matrix, pos1, pos2, chr1, chr2)
            ## for local alignment
            tmp_dir, tmp_score = local_alignment(score_matrix, pos1, pos2, chr1, chr2)
            
            score_matrix[pos1][pos2] = tmp_score
            dir_list[(pos1,pos2)] = tmp_dir
            if max_score < tmp_score:
                max_score = tmp_score
    return max_score

seq_human = seq_list['HUMAN']
for tmp_h in seq_list.keys():
    tmp_seq = seq_list[tmp_h]
    print("HUMAN-%s: %d"%(tmp_h, align_pairwise(seq_human, tmp_seq)))

HUMAN-HUMAN: 589
HUMAN-MOUSE: 433
HUMAN-CHICK: 329
HUMAN-XENLA: 292
HUMAN-DANRE: 199
